Simple reinforcement learning algorithm for learning tic-tac-toe
Use the update rule: V(s) = V(s) + alpha*(V(s') - V(s))
Use the epsilon-greedy policy:
action|s = argmax[over all actions possible from state s]{ V(s) }  if rand > epsilon
action|s = select random action from possible actions from state s if rand < epsilon

### INTERESTING THINGS TO TRY:
Currently, both agents use the same learning strategy while they play against each other.
What if they have different learning rates?
What if they have different epsilons? (probability of exploring)
Who will converge faster?
What if one agent doesn't learn at all?
Poses an interesting philosophical question: If there's no one around to challenge you,
can you reach your maximum potential?


In [134]:
from __future__ import print_function, division
# __future__ is a pseudo-module which programmers can use to
# enable new language features which are not compatible with the current interpreter.
from builtins import range, input
import numpy as np
import matplotlib.pyplot as plt
LENGTH = 3

In [135]:
#Agent
class Agent:
    def __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps # The probobility of choosing the not greedy path
        self.alpha = alpha # The learning rate - the weight of the current value vs the weight of the next steps values
        self.verbose = False #Somehow related to logging
        self.state_history = []
    
    def set_V(self, V):
        self.V = V
    
    def set_symbol(self, symbol):
        self.symbol = symbol
    
    def set_verbose(self, verbose):
        #if true  - prints values for every position in the board
        self.verbose = verbose
        
    def reset_history(self):
        self.state_history = []

    def take_action(self,env):
        r = np.random.rand()
        best_state = None
        
        if r < self.eps: 
        # In case we choose a random move - probobility of epsilon or less
            if self.verbose == True:
                print("Taking a random action")
            possible_moves = []
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i,j))
            next_move = possible_moves[np.random.choice(len(possible_moves))]
            
        else:
        # In case we choose a greedy move - probobility of more than epsilon
        #choosing the best action based on current values and states. looping through all possible moves to find max value
            pos2value={}#used for debugging
            next_move = None
            best_value = -1
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                    # Finding the best state moving to
                        env.board[i,j] = self.symbol
                        state = env.get_state()
                        env.board[i,j] = 0 
                        #returning the board to his previous state before assigning a sign to the board
                        pos2value[(i,j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i,j)
            if self.verbose == True:
                print("Taking a greedy action")
                for i in range(LENGTH):
                    print("------------------")
                    for j in range(LENGTH):
                        if env.is_empty(i, j):
                            # print the value
                            print(" %.2f|" % pos2value[(i,j)], end="")
                        else:
                            print("  ", end="")
                            if env.board[i,j] == env.x:
                                print("x  |", end="")
                            elif env.board[i,j] == env.o:
                                print("o  |", end="")
                            else:
                                print("   |", end="")
                        print("")
                    print("------------------")
            #making the move
            env.board[next_move[0],next_move[1]] = self.symbol
            
        
    def update_state_history(self, s):
    # s is env.get_state()
        self.state_history.append(s)
    
    def update(self,env):
        reward = env.reward(self.symbol)
        target = reward
        for prev in reversed(self.state_history):
            value = self.V[prev] + self.alpha*(target - self.V[prev])
            self.V[prev] = value
            target = value
        self.reset_history()
    
    
        

In [136]:
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH,LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_of_states = 3**(LENGTH * LENGTH)
        
    def is_empty(self, i, j):
        return self.board[i,j] == 0
    
    def reward(self, symbol):
        if self.game_over == True and self.winner == symbol:
            return 1
        else:
            return 0
    
    def get_state(self):
        k = 0
        h = 0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i,j] == 0:
                    v = 0
                elif self.board[i,j] == self.x:
                    v = 1
                elif self.board[i,j] == self.o:
                    v = 2
                h += (3**k) * v
                k += 1 
                # (0,0) -> k =0, (0,1) -> k=1, (1,0) -> k = 3, (2,2) -> k = 8
        return h
    
    def game_over(self, force_recalculate=False):
    # returns true if game over (a player has won or it's a draw)
    # otherwise returns false
    # also sets 'winner' instance variable and 'ended' instance variable
        if not force_recalculate and self.ended:
            return self.ended
        
        #Check rows
        for i in range(LENGTH):
            for player in (self.x,self.o):
                if self.board[i].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        #Check columns
        for j in range(LENGTH):
            for player in (self.x,self.o):
                if self.board[:,j].sum() == player*LENGTH:
                    self.winner = player
                    self.ended = True
                    return True
        
        # check diagonals
        for player in (self.x, self.o):
            # top-left -> bottom-right diagonal
            if self.board.trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True
            # top-right -> bottom-left diagonal
            if np.fliplr(self.board).trace() == player*LENGTH:
                self.winner = player
                self.ended = True
                return True

        # check if draw
        if np.all((self.board == 0) == False):
            # winner stays None
            self.winner = None
            self.ended = True
            return True

        # game is not over
        self.winner = None
        return False
    
    
    def is_draw(self):
        return self.ended and self.winner is None

  
    def draw_board(self):
        for i in range(LENGTH):
            print("-------------")
            for j in range(LENGTH):
                print("  ", end="")
                if self.board[i,j] == self.x:
                    print("x ", end="")
                elif self.board[i,j] == self.o:
                    print("o ", end="")
                else:
                    print("  ", end="")
                print("")
        print("-------------")

In [137]:
class Human:
    def __init__(self):
        pass
    
    def set_symbol(self, symbol):
        self.symbol = symbol
    # recursive function that will return all
    # possible states (as ints) and who the corresponding winner is for those states (if any)
    # (i, j) refers to the next cell on the board to permute (we need to try -1, 0, 1)
    # impossible games are ignored, i.e. 3x's and 3o's in a row simultaneously
    # since that will never happen in a real gamesymbol
        
    def take_action(self, env):
        while True:
            # break when an ileagel move was made
            move = input("Enter the cordinates of your chosen move")
            i = int(move[0])
            j = int(move[2])
            if (env.is_empty(i,j)):
                env.board[i,j] = self.symbol
                break
    
    def update(self, env):
        pass
    
    def update_state_history(self,state):
        pass
    
    
                
            

In [138]:
# recursive function that will return all
# possible states (as ints) and who the corresponding winner is for those states (if any)
# (i, j) refers to the next cell on the board to permute (we need to try -1, 0, 1)
# impossible games are ignored, i.e. 3x's and 3o's in a row simultaneously
# since that will never happen in a real game

def get_state_hash_and_winner(env, i=0, j=0):
  results = []

  for v in (0, env.x, env.o):
    env.board[i,j] = v # if empty board it should already be 0
    if j == 2:
      # j goes back to 0, increase i, unless i = 2, then we are done
      if i == 2:
        # the board is full, collect results and return
        state = env.get_state()
        ended = env.game_over(force_recalculate=True)
        winner = env.winner
        results.append((state, winner, ended))
      else:
        results += get_state_hash_and_winner(env, i + 1, 0)
    else:
      # increment j, i stays the same
      results += get_state_hash_and_winner(env, i, j + 1)

  return results


In [139]:
def initialV_x(env, state_winner_triples):
  # initialize state values as follows
  # if x wins, V(s) = 1
  # if x loses or draw, V(s) = 0
  # otherwise, V(s) = 0.5
  V = np.zeros(env.num_of_states)
  for state, winner, ended in state_winner_triples:
    if ended:
      if winner == env.x:
        v = 1
      else:
        v = 0
    else:
      v = 0.5
    V[state] = v
  return V


def initialV_o(env, state_winner_triples):
  # this is (almost) the opposite of initial V for player x
  # since everywhere where x wins (1), o loses (0)
  # but a draw is still 0 for o
  V = np.zeros(env.num_of_states)
  for state, winner, ended in state_winner_triples:
    if ended:
      if winner == env.o:
        v = 1
      else:
        v = 0
    else:
      v = 0.5
    V[state] = v
  return V

In [140]:

def play_game(p1, p2, env, draw=False):
  # loops until the game is over
  current_player = None
  while not env.game_over():
    # alternate between players
    # p1 always starts first
    if current_player == p1:
      current_player = p2
    else:
      current_player = p1

    # draw the board before the user who wants to see it makes a move
    if draw:
      if draw == 1 and current_player == p1:
        env.draw_board()
      if draw == 2 and current_player == p2:
        env.draw_board()

    # current player makes a move
    current_player.take_action(env)

    # update state histories
    state = env.get_state()
    p1.update_state_history(state)
    p2.update_state_history(state)

  if draw:
    env.draw_board()

  # do the value function update
  p1.update(env)
  p2.update(env)


if __name__ == '__main__':
  # train the agent
  p1 = Agent()
  p2 = Agent()

  # set initial V for p1 and p2
  env = Environment()
  state_winner_triples = get_state_hash_and_winner(env)


  Vx = initialV_x(env, state_winner_triples)
  p1.set_V(Vx)
  Vo = initialV_o(env, state_winner_triples)
  p2.set_V(Vo)

  # give each player their symbol
  p1.set_symbol(env.x)
  p2.set_symbol(env.o)

  T = 10000
  for t in range(T):
    if t % 200 == 0:
      print(t)
    play_game(p1, p2, Environment())

  # play human vs. agent
  # do you think the agent learned to play the game well?
  human = Human()
  human.set_symbol(env.o)
  while True:
    p1.set_verbose(True)
    play_game(p1, human, Environment(), draw=2)
    # I made the agent player 1 because I wanted to see if it would
    # select the center as its starting move. If you want the agent
    # to go second you can switch the human and AI.
    answer = input("Play again? [Y/n]: ")
    if answer and answer.lower()[0] == 'n':
      break


0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
Taking a greedy action
------------------
 0.19|
 0.19|
 0.19|
------------------
------------------
 0.19|
 0.19|
 0.22|
------------------
------------------
 0.19|
 0.19|
 0.19|
------------------
-------------
    
    
    
-------------
    
    
  x 
-------------
    
    
    
-------------
Enter the cordinates of your chosen move1,1
Taking a greedy action
------------------
 0.18|
 0.19|
 0.17|
------------------
------------------
 0.18|
  o  |
  x  |
------------------
------------------
 0.18|
 0.17|
 0.18|
------------------
-------------
    
  x 
    
-------------
    
  o 
  x 
-------------
    
    
    
-------------
Enter the cordinates of your chosen move1,1
Enter the cordinates of your chosen move2,2
Taking a greedy action


KeyboardInterrupt: 

In [146]:
        for i in range(LENGTH):
            print("-------------")
            for j in range(LENGTH):
                print("  ", end="")
                if Environment.board[i,j] == -1:
                    print("x ", end="")
                elif Environment.board[i,j] == 1:
                    print("o ", end="")
                else:
                    print("  ", end="")
                print("")
        print("-------------")

-------------
  

AttributeError: type object 'Environment' has no attribute 'board'